In [1]:
using Plots
using DelimitedFiles
using Quadrature

In [2]:
function piece_wise_linear(x_interval, y_interval, x) #these are the intervals in the x and y, each interval will have a lin fuunction
    if x < x_interval[1,1]
        my_i = 1;
    end
    if x >= x_interval[end, 2]
        my_i = size(x_interval,1);
    end
    for i = 1:size(x_interval, 1)
        if x_interval[i,1] <= x && x < x_interval[i, 2]
            my_i = i;
        end
    end
    a = (x - x_interval[my_i, 1])/(x_interval[my_i,2] - x_interval[my_i,1]);
    return a * y_interval[my_i,2] + (1 - a) * y_interval[my_i,1];
end

piece_wise_linear (generic function with 1 method)

In [3]:
function piece_wise_linear_csound(ρ, par)
    return piece_wise_linear(par[1], par[2], ρ) #par is an array with x_int and y_int
end

function integrate_EOS(ρ, der, par) #pressure comes from here
    prob = QuadratureProblem(der, 0.0, ρ, par)
    sol = solve(prob, QuadGKJL())#,reltol = 1e-10)
    return sol[1]
end

integrate_EOS (generic function with 1 method)

In [4]:
#DNS(CMF) shown here


mₙ = 938.918713;
nb = readdlm("eos.nb DNS(CMF)", skipstart = 2);
data = readdlm("eos.thermo DNS(CMF)", skipstart = 1);
Q1 = data[:,4];
Q7 = data[:,10];
P = Q1.*nb;
ρ = (Q7 .+ 1) .* (nb .* mₙ);

dP = P[2:end] - P[1:end-1];
dρ = ρ[2:end] - ρ[1:end-1];

sound_speed = (abs.(dP ./ dρ));
geo_conv = (1.6191004251588869e-18)*(1782700000000.0002);

ρ = ρ .*geo_conv;
P = P .*geo_conv;

ρ_cut = [0, 0.00138, 0.00142, 0.00393, 0.0178]; #these are the cut off densities
indices = [argmin(abs.(ρ .- x))[1] for x in ρ_cut]; #this gives you the first number of each pair

x_interval = [0 ρ[indices[1]]];
for i = 1:size(ρ_cut,1) - 1
    x_interval = vcat(x_interval,[ρ[indices[i]] ρ[indices[i + 1]]]);
end


y_interval = [0 sound_speed[indices[1]]];
for i = 1:size(ρ_cut,1) - 1
    y_interval = vcat(y_interval,[sound_speed[indices[i]] sound_speed[indices[i + 1]]]);
end

print(size(x_interval))
print(size(y_interval))

x = ρ[1:end - 1];
y = zeros(size(x,1));
for i = 1:size(x,1) #for i = 1 to 17801
    y[i] = piece_wise_linear(x_interval, y_interval, x[i]);
end

plotlyjs()

plot(x,y, label = "Piece-Wise Code", xlabel = "ρ", ylabel = "cₛ²")
plot!(ρ[1:end - 1], sound_speed, label = "DNS(CMF)")

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-16256211317880361204\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-6187127805067325173\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-16256211317880361204\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

(5, 2)(5, 2)

In [5]:
println(x_interval)
println(y_interval)

[0.0 2.683985562445732e-10; 2.683985562445732e-10 0.001387090243627891; 0.001387090243627891 0.0014224520180366636; 0.0014224520180366636 0.003935205491747961; 0.003935205491747961 0.01783099866626234]
[0.0 0.0003043663708371769; 0.0003043663708371769 0.34006522767154; 0.34006522767154 0.33222981114647837; 0.33222981114647837 0.500866829421404; 0.500866829421404 0.6223683933290903]


In [6]:
#DNS(CMF) shown here

plot(ρ, P, xaxis =  ("ρ", :log), yaxis = "P", title = "Density v Pressure", label = " CompOSE")

par = [x_interval, y_interval];

P_fit = zeros(size(ρ,1));
for i = 1:size(ρ,1)
    P_fit[i] = integrate_EOS(ρ[i], piece_wise_linear_csound, par)
end

plot!(ρ, P_fit, xaxis = :log, yaxis = :log, label = "Our Fit")

In [7]:
#BFH(QHC19-A) shown here


mₙ = 939.565000;
nb = readdlm("eos.nb BFH(QHC19-A)", skipstart = 2);
data = readdlm("eos.thermo BFH(QHC19-A)", skipstart = 1);
Q1 = data[:,4];
Q7 = data[:,10];
P = Q1.*nb;
ρ = (Q7 .+ 1) .* (nb .* mₙ);

dP = P[2:end] - P[1:end-1];
dρ = ρ[2:end] - ρ[1:end-1];

sound_speed = (abs.(dP ./ dρ));

geo_conv = (1.6191004251588869e-18)*(1782700000000.0002);
ρ = ρ .*geo_conv;


#ρ_cut = [0, 0.001459, 0.002352, 0.002653, 0.007690]; #these are the cut off densities,4.41e-12, 2.31e-9, 45.10e-6
#ρ_cut = [0, 0.001344, 0.002120, 0.002391, 0.006824];
#ρ_cut = [0, 0.001198, 0.001901, 0.002158, 0.006008];
ρ_cut = [0, 0.001072, 0.001700, 0.001936, 0.005619];


indices = [argmin(abs.(ρ .- x))[1] for x in ρ_cut]; #this gives you the first number of each pair

x_interval = [0 ρ[indices[1]]];
for i = 1:size(ρ_cut,1) - 1
    x_interval = vcat(x_interval,[ρ[indices[i]] ρ[indices[i + 1]]]);
end


y_interval = [0 sound_speed[indices[1]]];
for i = 1:size(ρ_cut,1) - 1
    y_interval = vcat(y_interval,[sound_speed[indices[i]] sound_speed[indices[i + 1]]]);
end

print(size(x_interval))
print(size(y_interval))

x = ρ[1:end - 1];
y = zeros(size(x,1));
for i = 1:size(x,1) #for i = 1 to 17801
    y[i] = piece_wise_linear(x_interval, y_interval, x[i]);
end

plotlyjs()

plot(x,y, label = "Our Sequence", title = "BFH(QHC19-A)", xlabel = "ρ (M⊙⁻²)", ylabel = "cₛ² (c²)", linewidth = 1.5)
plot!(ρ[1:end - 1], sound_speed, label = "CompOSE Sequence", linewidth = 1.5)

(5, 2)(5, 2)

In [8]:
#BFH(QHC19-A) shown here

println(x_interval)
println(y_interval)

[0.0 2.121710214285006e-13; 2.121710214285006e-13 0.0010727599415600536; 0.0010727599415600536 0.001700692692771716; 0.001700692692771716 0.0019366938762646454; 0.0019366938762646454 0.005619849619591004]
[0.0 1.096376754968612e-5; 1.096376754968612e-5 0.2547646587774711; 0.2547646587774711 0.3853417266187098; 0.3853417266187098 0.38445445767621794; 0.38445445767621794 0.653305141330959]


In [9]:
#BFH(QHC19-A) shown here

plot(ρ, P, xaxis =  ("ρ", :log), yaxis = "P", title = "Density v Pressure", label = " CompOSE")

par = [x_interval, y_interval];

P_fit = zeros(size(ρ,1));
for i = 1:size(ρ,1)
    P_fit[i] = integrate_EOS(ρ[i], piece_wise_linear_csound, par)
end

plot!(ρ, P_fit, xaxis = :log, yaxis = :log, label = "Our Fit")

In [10]:
#BHK(QHC18) shown here


mₙ = 939.565000;
nb = readdlm("eos.nb BHK(QHC18)", skipstart = 2);
data = readdlm("eos.thermo BHK(QHC18)", skipstart = 1);
Q1 = data[:,4];
Q7 = data[:,10];
P = Q1.*nb;
ρ = (Q7 .+ 1) .* (nb .* mₙ);

dP = P[2:end] - P[1:end-1];
dρ = ρ[2:end] - ρ[1:end-1];

sound_speed = (abs.(dP ./ dρ));
geo_conv = (1.6191004251588869e-18)*(1782700000000.0002);
ρ = ρ .*geo_conv;


#ρ_cut = [0 , 0.001701, 0.002367, 0.002721, 0.005571]; #these are the cut off densities
#ρ_cut = [0 , 0.001530, 0.002130, 0.002448, 0.005013]; #these are the cut off densities
#ρ_cut = [0 , 0.001377, 0.001917, 0.002203, 0.004511]; #these are the cut off densities
ρ_cut = [0 , 0.001239, 0.001725, 0.001982, 0.004059]; #these are the cut off densities

indices = [argmin(abs.(ρ .- x))[1] for x in ρ_cut]; #this gives you the first number of each pair

x_interval = [0 ρ[indices[1]]];
for i = 1:size(ρ_cut,1) - 1
    x_interval = vcat(x_interval,[ρ[indices[i]] ρ[indices[i + 1]]]);
end


y_interval = [0 sound_speed[indices[1]]];
for i = 1:size(ρ_cut,1) - 1
    y_interval = vcat(y_interval,[sound_speed[indices[i]] sound_speed[indices[i + 1]]]);
end

print(size(x_interval))
print(size(y_interval))

x = ρ[1:end - 1];
y = zeros(size(x,1));
for i = 1:size(x,1) #for i = 1 to 17801
    y[i] = piece_wise_linear(x_interval, y_interval, x[i]);
end

plotlyjs()

plot(x,y, label = "Piece-Wise Code", xlabel = "ρ", ylabel = "cₛ²")
plot!(ρ[1:end - 1], sound_speed, label = "BHK(QHC18)" )

(5, 2)(5, 2)

In [11]:
#BHK(QHC18) shown here

println(x_interval)
println(y_interval)

[0.0 2.332827999180845e-13; 2.332827999180845e-13 0.0012318162648510052; 0.0012318162648510052 0.001728644303027397; 0.001728644303027397 0.001978768496534892; 0.001978768496534892 0.004109296572171727]
[0.0 1.1297422315066974e-5; 1.1297422315066974e-5 0.2915869442881261; 0.2915869442881261 0.587415540540544; 0.587415540540544 0.5491339696005639; 0.5491339696005639 0.6458839150227601]


In [12]:
#BHK(QHC18) shown here

plot(ρ, P, xaxis =  ("ρ", :log), yaxis = "P", title = "Density v Pressure", label = " CompOSE")

par = [x_interval, y_interval];

P_fit = zeros(size(ρ,1));
for i = 1:size(ρ,1)
    P_fit[i] = integrate_EOS(ρ[i], piece_wise_linear_csound, par)
end

plot!(ρ, P_fit, xaxis = :log, yaxis = :log, label = "Our Fit")

In [13]:
#RG(SkI3) shown here


mₙ = 939.565330;
nb = readdlm("eos.nb RG(SkI3)", skipstart = 2);
data = readdlm("eos.thermo RG(SkI3)", skipstart = 1);
Q1 = data[:,4];
Q7 = data[:,10];
P = Q1.*nb;
ρ = (Q7 .+ 1) .* (nb .* mₙ);

dP = P[2:end] - P[1:end-1];
dρ = ρ[2:end] - ρ[1:end-1];

sound_speed = (abs.(dP ./ dρ));
geo_conv = (1.6191004251588869e-18)*(1782700000000.0002);
ρ = ρ .*geo_conv;


#ρ_cut = [0 , 0.002578, 0.004880, 0.007484, 0.009836]; #these are the cut off densities
#ρ_cut = [0 , 0.001002, 0.002473, 0.003406, 0.004660];
#ρ_cut = [0 , 0.0009018, 0.002277, 0.003055, 0.004159];
ρ_cut = [0 , 0.0008116, 0.002049, 0.002749, 0.0037431];

indices = [argmin(abs.(ρ .- x))[1] for x in ρ_cut]; #this gives you the first number of each pair

x_interval = [0 ρ[indices[1]]];
for i = 1:size(ρ_cut,1) - 1
    x_interval = vcat(x_interval,[ρ[indices[i]] ρ[indices[i + 1]]]);
end


y_interval = [0 sound_speed[indices[1]]];
for i = 1:size(ρ_cut,1) - 1
    y_interval = vcat(y_interval,[sound_speed[indices[i]] sound_speed[indices[i + 1]]]);
end

print(size(x_interval))
print(size(y_interval))

x = ρ[1:end - 1];
y = zeros(size(x,1));
for i = 1:size(x,1) #for i = 1 to 17801
    y[i] = piece_wise_linear(x_interval, y_interval, x[i]);
end

plotlyjs()

plot(x,y, label = "Piece-Wise Code", xlabel = "ρ", ylabel = "cₛ²")
plot!(ρ[1:end - 1], sound_speed, label = "RG(SkI3)", ylims = (0.0, 1.0))

(5, 2)(5, 2)

In [14]:
#RG(SkI3) shown here

println(x_interval)
println(y_interval)

[0.0 2.6860969249939775e-10; 2.6860969249939775e-10 0.0008115119654457464; 0.0008115119654457464 0.002058120589590364; 0.002058120589590364 0.0027466685608737353; 0.0027466685608737353 0.003719766464565832]
[0.0 0.0003114593806787249; 0.0003114593806787249 0.2069894755658828; 0.2069894755658828 0.5871816110996051; 0.5871816110996051 0.7372991600681386; 0.7372991600681386 0.8917380491848568]


In [15]:
#RG(SkI3) shown here

plot(ρ, P, xaxis =  ("ρ", :log), yaxis = "P", title = "Density v Pressure", label = " CompOSE")

par = [x_interval, y_interval];

P_fit = zeros(size(ρ,1));
for i = 1:size(ρ,1)
    P_fit[i] = integrate_EOS(ρ[i], piece_wise_linear_csound, par)
end

plot!(ρ, P_fit, xaxis = :log, yaxis = :log, label = "Our Fit")